In [1]:
!pip install transformers --quiet

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 3.3 MB 45.2 MB/s 
     |████████████████████████████████| 61 kB 468 kB/s 


In [2]:
!pip install datasets --quiet

     |████████████████████████████████| 306 kB 5.4 MB/s 
     |████████████████████████████████| 243 kB 43.6 MB/s 
     |████████████████████████████████| 1.1 MB 37.4 MB/s 
     |████████████████████████████████| 132 kB 45.4 MB/s 
     |████████████████████████████████| 160 kB 45.8 MB/s 
     |████████████████████████████████| 271 kB 48.0 MB/s 
     |████████████████████████████████| 192 kB 32.6 MB/s 


In [3]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_train.tsv


--2022-01-01 19:05:24--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 364313 (356K) [text/plain]
Saving to: ‘BASA_cricket_train.tsv’

BASA_cricket_train. 100%[===================>] 355.77K  --.-KB/s    in 0.04s   

2022-01-01 19:05:25 (8.87 MB/s) - ‘BASA_cricket_train.tsv’ saved [364313/364313]



In [4]:
!wget https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_test.tsv

--2022-01-01 19:05:25--  https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66936 (65K) [text/plain]
Saving to: ‘BASA_cricket_test.tsv’

BASA_cricket_test.t 100%[===================>]  65.37K  --.-KB/s    in 0.01s   

2022-01-01 19:05:25 (5.78 MB/s) - ‘BASA_cricket_test.tsv’ saved [66936/66936]



In [5]:
import pandas as pd

train_df = pd.read_csv("BASA_cricket_train.tsv", sep = "\t")
test_df = pd.read_csv("BASA_cricket_test.tsv", sep = "\t")

In [6]:
train_df.tail(10)

,id,text,class_label
2075,1176,আমাদের ব্যাটসম্যানরা বাজে শটস খেলে আউট হয়েছে। ...,positive
2076,1511,সময় এসেছে রাজাকার পিচ কিউরেটর কে পাছায় লাথি দি...,negative
2077,2463,ক্রিকেটারদের প্রতিযোগিতামূলক ক্রিকেট সম্বন্ধে ...,negative
2078,1038,"দল জেতায় মুশি ক্রেডিট ম্যাসের,দল জেতায় রিয়াদ আ...",negative
2079,2911,তাদের স্পীন এটাক যে আমাদের থেকে ভালো তা কি এই ...,negative
2080,1137,বিদেশী ৫ জন এক ম্যাচে ঠিকই আছে।,neutral
2081,511,দ্রুত তাকে জাতীয়দলে সুযোগ দিয়ে ক্যারিয়ার ধংস্ব...,negative
2082,2611,"ওই নিম্ন মানের পাড়ার ক্রিকেট, ঘরোয়া লীগ, বিপিএ...",negative
2083,2613,নিলয় সবজায়গায় দেখি মাশরাফি নিয়ে বাজে কমেন্ট কর...,negative
2084,908,দ্বিতীয় ইনিংসে লিটন দাসের ট্রিপল সেঞ্চুরি চাই,positive


In [7]:
len(train_df)

2085

In [8]:
train_df["class_label"].value_counts()

negative    1515
positive     376
neutral      194
Name: class_label, dtype: int64

In [9]:
def class_to_numeric(df_column):
  if df_column == "neutral":
    return 0
  elif df_column == "positive":
    return 1
  elif df_column == "negative":
    return 2

In [10]:
train_labels = pd.Series(train_df["class_label"].apply(class_to_numeric))

In [11]:
len(train_labels) ==  len(train_df)

True

In [12]:
train_df.loc[1]

id                                                           955
text           মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...
class_label                                             positive
Name: 1, dtype: object

In [13]:
train_df["labels"] = train_labels
train_df.head()

,id,text,class_label,labels
0,2760,আফতাব আপনি ভালো আছেন?,neutral,0
1,955,মমিনুল হক আর মুশফিক দুজনেই নিজেদের উইকেট জঘন্য...,positive,1
2,43,মিরপুর এর পিচ এ যে কেও বোলিং করতে পারে,neutral,0
3,2211,মুদ্রার ঐ পিঠ দেখা হয়ে গেল বাংলাদেশের.দেশের মা...,negative,2
4,2490,শুভকামনা রাজ্জাক ভাই।,negative,2


In [14]:
test_labels = pd.Series(test_df["class_label"].apply(class_to_numeric))
test_df["labels"] = test_labels
test_df.head()

,id,text,class_label,labels
0,2754,ইমরুল বাদে বাকি তিনজনের আউট মেনেনিতে পারছিনা ?,negative,2
1,2746,"আফতাব আহমেদ, আপনার সেই ৬টা এখনো ভুলতে পারিনি।",negative,2
2,21,জরিমানা করা হউক। ৩ মাসের বেতন কর্তন।,negative,2
3,2143,মামুর পোলারা কোচের বদ নাম করো এবার কোচ কি জিনি...,negative,2
4,426,হাথুরে বেটা কত্তটা যে খারাপ সিদ্ধান্ত বাংলাদেশ...,negative,2


In [15]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
train_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 2085
})

In [16]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset

Dataset({
    features: ['id', 'text', 'class_label', 'labels'],
    num_rows: 380
})

In [17]:
from transformers import AutoTokenizer 
checkpoint = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

In [19]:
def tokenize_function(example):
  return tokenizer(
      example["text"],
      truncation = True
  )

In [20]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched = True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched = True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
tokenized_train_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'class_label': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [22]:
def process_tokenize(tokenized_dataset):
  tokenized_dataset = tokenized_dataset.remove_columns(["id","text","class_label"])
  tokenized_dataset = tokenized_dataset.with_format("torch")
  return tokenized_dataset

In [23]:
tokenized_train_dataset = process_tokenize(tokenized_train_dataset)
tokenized_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 2085
})

In [24]:
tokenized_test_dataset = process_tokenize(tokenized_test_dataset)
tokenized_test_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids'],
    num_rows: 380
})

In [25]:
tokenized_test_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([  101,   638, 28558, 28560, 28569, 28561,   654, 28566, 28553, 28570,
           654, 28566, 28544, 28567,   648, 28567, 28555, 28549, 28555, 28570,
         28560,   100,   655, 28570, 28555, 28570, 28555, 28567, 28551, 28570,
           653, 28566, 28560, 28548, 28567, 28555, 28566,   136,   102]),
 'labels': tensor(2),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [26]:
tokenized_test_dataset[0]['attention_mask'].shape

torch.Size([39])

In [27]:
tokenized_test_dataset[1]["attention_mask"].shape

torch.Size([30])

In [28]:
tokenized_test_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [29]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train_dataset,batch_size = 8, shuffle = True, collate_fn = data_collator)
eval_dataloader = DataLoader(tokenized_test_dataset,batch_size = 8, collate_fn = data_collator)

In [30]:
train_dataloader

In [31]:
len(train_dataloader)

261

In [32]:
for batch in eval_dataloader:
  print(batch)
  break

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1,

In [33]:
for batch in train_dataloader:
  print(batch.keys())
  break

dict_keys(['attention_mask', 'input_ids', 'labels', 'token_type_ids'])


In [34]:
for batch in train_dataloader:
  print({k: v.shape for k  , v in batch.items() })
  break

{'attention_mask': torch.Size([8, 54]), 'input_ids': torch.Size([8, 54]), 'labels': torch.Size([8]), 'token_type_ids': torch.Size([8, 54])}


In [35]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [36]:
outputs = model(**batch)
print(outputs)

SequenceClassifierOutput(loss=tensor(1.4127, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0530, -0.1084, -0.7305],
        [-0.3521, -0.0467, -0.5644],
        [ 0.0008, -0.1532, -0.7094],
        [-0.2806, -0.0375, -0.6297],
        [-0.0119, -0.1181, -0.6961],
        [-0.1675, -0.1452, -0.6572],
        [ 0.0369, -0.2147, -0.7119],
        [-0.0779, -0.1591, -0.6726]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [37]:
outputs.loss , outputs.logits.shape

(tensor(1.4127, grad_fn=<NllLossBackward0>), torch.Size([8, 3]))

In [38]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr = 5e-5)

In [39]:
from transformers import get_scheduler

num_epochs = 4
num_training_steps = num_epochs * len(train_dataset)
lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)
print(num_training_steps)

8340


In [40]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [41]:
# import numpy as np

# def eval_model(model, data_loader, device, n_examples):
#   model = model.eval()
#   losses = []
#   correct_predictions = 0

#   with torch.no_grad():
#    for d in data_loader:
#      targets = d["labels"]#.to(device)

#      outputs = model(**d)
#      preds = torch.argmax(outputs, dim=-1)

#      loss = outputs.loss
     

#      correct_predictions += torch.sum(preds == targets)
#      losses.append(loss.item())

#   return correct_predictions.double() / n_examples, np.mean(losses)

In [42]:
batch.keys()

dict_keys(['attention_mask', 'input_ids', 'labels', 'token_type_ids'])

In [43]:
import numpy as np

def eval_model(model, data_loader, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0

  
  for d in data_loader:
     d = {k: v.to(device) for k, v in d.items()}
     targets = d["labels"]

     with torch.no_grad():

      outputs = model(**d)
     logits = outputs.logits
     preds = torch.argmax(logits, dim=-1)

     loss = outputs.loss
     

     correct_predictions += torch.sum(preds == targets)
     losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)


In [44]:
len(eval_dataloader)

48

In [45]:

eval_model(model,eval_dataloader,len(eval_dataloader)*8)

(tensor(0.1302, device='cuda:0', dtype=torch.float64), 1.3575431654850643)

In [46]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/8340 [00:00<?, ?it/s]

In [47]:
eval_model(model,eval_dataloader,len(eval_dataloader)*8)

(tensor(0.7109, device='cuda:0', dtype=torch.float64), 0.7657027132809162)